In [ ]:
import ccxt
import pandas as pd

# Initialize the specific exchange 
EXCHANGE_NAME = "kraken"
EXCHANGE = getattr(ccxt, EXCHANGE_NAME)()  # i.e. ccxt.kraken()

# Fetch all markets and filter spot pairs
markets = EXCHANGE.load_markets()  # i.e. kraken.load_markets() 
coin_spot_pairs = [
    market for trade_pair, market in markets.items()
    if market.get('spot', True) and trade_pair[-3:] == "USD"  # Identifies only USD pairs
]


# Define the timeframe and limit for OHLC data
timeframe = '1h'  # Example: Hourly data
limit = 1       # Number of data points to fetch

# Fetch OHLC data for the all  pairs
ohlc_data = {}
for market in coin_spot_pairs:
    symbol = market['symbol']
    try:
        # Fetch OHLCV data for the current pair
        ohlcv = EXCHANGE.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
        # Convert to a DataFrame
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        ohlc_data[symbol] = df
        ohlc_data[symbol]['usd_volume'] = ohlc_data[symbol]['close'] * ohlc_data[symbol]['volume']
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Example: Display data for the first pair
if ohlc_data:
    first_pair = list(ohlc_data.keys())[0]
    print(f"OHLC data for {first_pair}:")
    print(ohlc_data[first_pair])


OHLC data for 1INCH/USD:
            timestamp   open   high    low  close      volume  usd_volume
0 2024-12-07 11:00:00  0.529  0.533  0.529  0.533  786.802149  419.365546


In [20]:
# get 200 largest pairs by volume
pair_vol_list = [[pair, data['usd_volume'].values[0] ] for pair, data in ohlc_data.items()]
sorted_pair_vol_list = sorted(pair_vol_list, key=lambda x: x[1], reverse=True)
sorted_pairs = [x[0] for x in sorted_pair_vol_list]

# save pairs in order by 24hr usd volume
NUM_TICKERS = 500
df = pd.Series(sorted_pairs[:NUM_TICKERS])
df.to_csv('pairs.csv', index=False)